<a href="https://colab.research.google.com/github/R1tsuko/big_data/blob/main/L1%20-%20Introduction%20to%20Apache%20Spark/BigData_Lab2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Настройка, установка и обработка текста

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
sc = spark.sparkContext
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving stations.csv to stations.csv


In [ ]:
warandpeace = sc.textFile("warandsociety.txt")
print(warandpeace.count())

nilFile = sc.textFile("nil")
print(nilFile.count())

12851


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/nil
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:297)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:239)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:325)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.api.python.PythonRDD.getPartitions(P

In [ ]:
print(warandpeace.take(10))
print(warandpeace.getNumPartitions())

['Лев Николаевич Толстой', 'Война и мир. Книга 1', '', 'Война и мир – 1', '', ' ', ' http://www.lib.ru', '', 'Аннотация ', '']
2


In [ ]:
linesWithWar = warandpeace.filter(lambda x: "война" in x)
linesWithWar.first()

"– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист) – я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите.] Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, [Я вижу, что я вас пугаю,] садитесь и рассказывайте."

In [ ]:
def time(f):
    import time
    t = time.process_time()
    f()
    print(f"Elapsed time: {int((time.process_time() - t)*1e9)} ns")

linesWithWar.cache()
time(lambda: linesWithWar.count() )
time(lambda: linesWithWar.count() )

Elapsed time: 6774192 ns
Elapsed time: 11663164 ns


In [ ]:
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wordCounts.saveAsTextFile("warandpeace_histogram.txt")

In [ ]:
!ls -l warandpeace_histogram.txt
!head -10 warandpeace_histogram.txt/*

total 64
-rw-r--r-- 1 root root 32428 Apr 22 10:59 part-00000
-rw-r--r-- 1 root root 32489 Apr 22 10:59 part-00001
-rw-r--r-- 1 root root     0 Apr 22 10:59 _SUCCESS
==> warandpeace_histogram.txt/part-00000 <==
('Еh', 1)
('mon', 6)
('prince.', 1)
('sont', 1)
('que', 9)
('des', 2)
('поместья,', 1)
('Buonaparte.', 1)
('Non,', 2)
('previens,', 1)

==> warandpeace_histogram.txt/part-00001 <==
('–', 92)
('bien,', 1)
('Genes', 1)
('et', 4)
('Lucques', 1)
('ne', 4)
('plus', 3)
('apanages,', 1)
('de', 13)
('la', 6)

==> warandpeace_histogram.txt/_SUCCESS <==


In [ ]:
import re
def dashrepl(matchobj):
    return re.sub(r'[^\w]', '', matchobj.group(0)).lower()
wordCounts = linesWithWar.flatMap(lambda line: line.split(" ")).map(lambda word: (re.sub(r"[A-Z,А-Я]+|['-]", dashrepl, word), 1)).reduceByKey(lambda a, b: a + b)
wordCounts.saveAsTextFile("warandpeace_histogram1.txt")
!ls -l warandpeace_histogram1.txt
!head -10 warandpeace_histogram1.txt/*

total 64
-rw-r--r-- 1 root root 30095 Apr 22 13:30 part-00000
-rw-r--r-- 1 root root 29044 Apr 22 13:30 part-00001
-rw-r--r-- 1 root root     0 Apr 22 13:30 _SUCCESS
==> warandpeace_histogram1.txt/part-00000 <==
('bien', 1)
('mon', 6)
('prince.', 1)
('genes', 1)
('lucques', 1)
('sont', 1)
('que', 9)
('des', 2)
('apanages', 1)
('buonaparte.', 1)

==> warandpeace_histogram1.txt/part-00001 <==
('–', 92)
('еh', 1)
('et', 4)
('ne', 4)
('plus', 5)
('поместья', 1)
('de', 14)
('la', 7)
('famille', 1)
('non', 2)

==> warandpeace_histogram1.txt/_SUCCESS <==


#### Операции с множествами

In [ ]:
a = sc.parallelize([1,2,3,4])
b = sc.parallelize([3,4,6,7])

In [ ]:
print(a.union(b).distinct().collect())
print(a.intersection(b).collect())
print(a.subtract(b).collect())

[4, 1, 2, 6, 3, 7]
[4, 3]
[1, 2]


def distinct(self: "RDD[T]", numPartitions: Optional[int] = None) -> "RDD[T]":
        
  
        return (
            self.map(lambda x: (x, None))
            .reduceByKey(lambda x, _: x, numPartitions)
            .map(lambda x: x[0])
        )

Упражнение. Найдите в исходном коде Spark определение функции distinct. Объясните почему реализация этой операции действительно убирает дубликаты.
Очень просто - элементы попадают на этап редьюс как ключи, каждый элемент в итоге дает одно значение

#### Общие переменные и аккумуляирующие переменные

In [ ]:
broadcastVar = sc.broadcast([1,2,3])
broadcastVar.value

[1, 2, 3]

In [ ]:
accum = sc.accumulator(0)
sc.parallelize([1,2,3,4]).foreach(lambda x: accum.add(x))
accum.value

10

In [ ]:
pair = ('a', 'b')

#### Топ 10 популярных номеров такси

In [ ]:
taxi = sc.textFile("nyctaxi.csv")
for t in taxi.take(5):
    print(t)
import itertools
taxi.mapPartitionsWithIndex(lambda idx, it:  itertools.islice(it,1,None) if (idx==0) else it  )

"_id","_rev","dropoff_datetime","dropoff_latitude","dropoff_longitude","hack_license","medallion","passenger_count","pickup_datetime","pickup_latitude","pickup_longitude","rate_code","store_and_fwd_flag","trip_distance","trip_time_in_secs","vendor_id"
"29b3f4a30dea6688d4c289c9672cb996","1-ddfdec8050c7ef4dc694eeeda6c4625e","2013-01-11 22:03:00",+4.07033460000000E+001,-7.40144200000000E+001,"A93D1F7F8998FFB75EEF477EB6077516","68BC16A99E915E44ADA7E639B4DD5F59",2,"2013-01-11 21:48:00",+4.06760670000000E+001,-7.39810790000000E+001,1,,+4.08000000000000E+000,900,"VTS"
"2a80cfaa425dcec0861e02ae44354500","1-b72234b58a7b0018a1ec5d2ea0797e32","2013-01-11 04:28:00",+4.08190960000000E+001,-7.39467470000000E+001,"64CE1B03FDE343BB8DFB512123A525A4","60150AA39B2F654ED6F0C3AF8174A48A",1,"2013-01-11 04:07:00",+4.07280540000000E+001,-7.40020370000000E+001,1,,+8.53000000000000E+000,1260,"VTS"
"29b3f4a30dea6688d4c289c96758d87e","1-387ec30eac5abda89d2abefdf947b2c1","2013-01-11 22:02:00",+4.07277180000000E+00

PythonRDD[133] at RDD at PythonRDD.scala:53

In [ ]:
taxiParse = taxi.map(lambda line: line.split(","))
taxiMedKey = taxiParse.map(lambda row: (row[6], 1))
taxiMedCounts = taxiMedKey.reduceByKey(lambda v1, v2: v1+v2)
top10 = taxiMedCounts.map(lambda x: x[::-1]).top(10)
for x in top10:
    print(x[::-1])

('"AB44AD9A03B7CFAF3925103BDCC0AF23"', 44)
('"71CACFBADF9568AAE88A843DB511D172"', 41)
('"6483B9BFCB216EC88986EA3AB13064E7"', 41)
('"4C73459B430339981D78795300433438"', 41)
('"67E71D24AF704D814A0A825005ADA72E"', 40)
('"02E5A4136FD0A775A023A005A4EABC62"', 40)
('"9DFBCD218E7116F34C044F0680A0FB8A"', 39)
('"8DEB70907D00AA1D7FF5E2683240549B"', 39)
('"7989C2AB3F345F4AB54D3CF1E0480D67"', 39)
('"6C9F67DF658DC5636F9E7752F203F70A"', 39)


In [ ]:
taxiMedCounts.cache()
time(lambda: taxiMedCounts.count())
time(lambda: taxiMedCounts.count())

Elapsed time: 5412668 ns
Elapsed time: 11958155 ns


#### Настройка способа хранения RDD

In [ ]:
from pyspark import StorageLevel
taxi.persist(storageLevel=StorageLevel.MEMORY_ONLY)

nyctaxi.csv MapPartitionsRDD[131] at textFile at NativeMethodAccessorImpl.java:0

#### 1. Проведите анализ данных велопарковок на языке Python в интерактивном режиме из Jupyter книг:

In [ ]:
tripData = sc.textFile("trips.csv")
# запомним заголовок, чтобы затем его исключить из данных
tripsHeader = tripData.first()
trips = tripData.filter(lambda row: row != tripsHeader).map(lambda row: row.split(",", -1))

stationData = sc.textFile("stations.csv")
stationsHeader = stationData.first()
stations = stationData.filter(lambda row: row != stationsHeader).map(lambda row: row.split(",", -1))

In [ ]:
list(enumerate(tripsHeader.split(",")))

[(0, 'id'),
 (1, 'duration'),
 (2, 'start_date'),
 (3, 'start_station_name'),
 (4, 'start_station_id'),
 (5, 'end_date'),
 (6, 'end_station_name'),
 (7, 'end_station_id'),
 (8, 'bike_id'),
 (9, 'subscription_type'),
 (10, 'zip_code')]

In [ ]:
list(enumerate(stationsHeader.split(",")))

[(0, 'id'),
 (1, 'name'),
 (2, 'lat'),
 (3, 'long'),
 (4, 'dock_count'),
 (5, 'city'),
 (6, 'installation_date')]

In [ ]:
trips.take(2)

[['4576',
  '63',
  '',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138']]

In [ ]:
stations.take(2)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013']]

In [ ]:
stationsIndexed = stations.keyBy(lambda station: station[0])
stationsIndexed.take(2)

[('2',
  ['2',
   'San Jose Diridon Caltrain Station',
   '37.329732',
   '-121.90178200000001',
   '27',
   'San Jose',
   '8/6/2013']),
 ('3',
  ['3',
   'San Jose Civic Center',
   '37.330698',
   '-121.888979',
   '15',
   'San Jose',
   '8/5/2013'])]

In [ ]:
tripsByStartTerminals = trips.keyBy(lambda trip: trip[4])
tripsByEndTerminals = trips.keyBy(lambda trip: trip[7])

In [ ]:
startTrips = stationsIndexed.join(tripsByStartTerminals)
endTrips = stationsIndexed.join(tripsByEndTerminals)

In [ ]:
print(startTrips.toDebugString().decode("utf-8"))

(5) PythonRDD[175] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[167] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[166] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[165] at join at <ipython-input-109-d9b5f12a0e8e>:1 []
    |  PythonRDD[164] at join at <ipython-input-109-d9b5f12a0e8e>:1 []
    |  UnionRDD[163] at union at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[161] at RDD at PythonRDD.scala:53 []
    |  stations.csv MapPartitionsRDD[156] at textFile at NativeMethodAccessorImpl.java:0 []
    |  stations.csv HadoopRDD[155] at textFile at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[162] at RDD at PythonRDD.scala:53 []
    |  trips.csv MapPartitionsRDD[153] at textFile at NativeMethodAccessorImpl.java:0 []
    |  trips.csv HadoopRDD[152] at textFile at NativeMethodAccessorImpl.java:0 []


In [ ]:
print(endTrips.toDebugString().decode("utf-8"))

(5) PythonRDD[176] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[174] at mapPartitions at PythonRDD.scala:145 []
 |  ShuffledRDD[173] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(5) PairwiseRDD[172] at join at <ipython-input-109-d9b5f12a0e8e>:2 []
    |  PythonRDD[171] at join at <ipython-input-109-d9b5f12a0e8e>:2 []
    |  UnionRDD[170] at union at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[168] at RDD at PythonRDD.scala:53 []
    |  stations.csv MapPartitionsRDD[156] at textFile at NativeMethodAccessorImpl.java:0 []
    |  stations.csv HadoopRDD[155] at textFile at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[169] at RDD at PythonRDD.scala:53 []
    |  trips.csv MapPartitionsRDD[153] at textFile at NativeMethodAccessorImpl.java:0 []
    |  trips.csv HadoopRDD[152] at textFile at NativeMethodAccessorImpl.java:0 []


In [ ]:
startTrips.count()

669959

In [ ]:
endTrips.count()

669959

In [ ]:
from pyspark.rdd import portable_hash

stationsIndexed.partitionBy(numPartitions=trips.getNumPartitions(), partitionFunc=lambda x: portable_hash(x[0]))

MapPartitionsRDD[182] at mapPartitions at PythonRDD.scala:145

In [ ]:
from typing import NamedTuple
from datetime import datetime

In [ ]:
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str

    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )
stationsInternal = stations.mapPartitions(initStation)
stationsInternal.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))

In [ ]:
def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str

    for trip in trips:
        try:
            yield Trip(
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            )
        except:
            pass
tripsInternal = trips.mapPartitions(initTrip)
tripsInternal.take(10)

[Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214'),
 Trip(trip_id=4251, duration=77, start_date=datetime.datetime(2013, 8, 29, 11, 29), start_station_name='San Jose City Hall', start_station_id=10, end_date=datetime.datetime(2013, 8, 29, 11, 30), end_station_name='San Jose City Hall', end_station_id='10', bike_id=26, subscription_type='Subscriber', zip_code='95060'),
 Trip(trip_id=4299, duration=83, start_date=datetime.datetime(2013, 8, 29, 12, 2), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 12, 4), end_station_name='Market at 10th', end_station_id='67', bike_id=319, subscription_type='Subscriber', zip_code='94103'),
 Trip(trip_id=4927, duration=103, s

In [ ]:
tripsByStartStation = tripsInternal.keyBy(lambda trip: trip.start_station_name)

In [ ]:
import numpy as np

avgDurationByStartStation = tripsByStartStation\
 .mapValues(lambda trip: trip.duration)\
 .groupByKey()\
 .mapValues(lambda trip_durations: np.mean(list(trip_durations)))

In [ ]:
%%time

avgDurationByStartStation.top(10, key=lambda x: x[1])

CPU times: user 138 ms, sys: 7.06 ms, total: 145 ms
Wall time: 21.2 s


[('University and Emerson', 7090.239417989418),
 ('California Ave Caltrain Station', 4628.005847953216),
 ('Redwood City Public Library', 4579.234741784037),
 ('Park at Olive', 4438.1613333333335),
 ('San Jose Civic Center', 4208.016938519448),
 ('Rengstorff Avenue / California Street', 4174.082373782108),
 ('Redwood City Medical Center', 3959.491961414791),
 ('Palo Alto Caltrain Station', 3210.6489815253435),
 ('San Mateo County Center', 2716.7700348432054),
 ('Broadway at Main', 2481.2537313432836)]

In [ ]:
def seqFunc(acc, duration):
    duration_sum, count = acc
    return (duration_sum + duration, count + 1)

def combFunc(acc1, acc2):
    duration_sum1, count1 = acc1
    duration_sum2, count2 = acc2
    return (duration_sum1+duration_sum2, count1+count2)

def meanFunc(acc):
    duration_sum, count = acc
    return duration_sum/count

avgDurationByStartStation2 = tripsByStartStation\
  .mapValues(lambda trip: trip.duration)\
  .aggregateByKey(
    zeroValue=(0,0),
    seqFunc=seqFunc,
    combFunc=combFunc)\
  .mapValues(meanFunc)

In [ ]:
%%time

avgDurationByStartStation2.top(10, key=lambda x: x[1])

CPU times: user 126 ms, sys: 9.56 ms, total: 135 ms
Wall time: 19.3 s


[('University and Emerson', 7090.239417989418),
 ('California Ave Caltrain Station', 4628.005847953216),
 ('Redwood City Public Library', 4579.234741784037),
 ('Park at Olive', 4438.1613333333335),
 ('San Jose Civic Center', 4208.016938519448),
 ('Rengstorff Avenue / California Street', 4174.082373782108),
 ('Redwood City Medical Center', 3959.491961414791),
 ('Palo Alto Caltrain Station', 3210.6489815253435),
 ('San Mateo County Center', 2716.7700348432054),
 ('Broadway at Main', 2481.2537313432836)]

In [ ]:
def earliestTrip(trips):
    if trips is None:
        return None
    if len(trips)==0:
        return trips
    trips = list(trips)
    min_date = trips[0].start_date
    min_trip = trips[0]
    for trip in trips[1:]:
        if min_date > trip.start_date:
            min_date = trip.start_date
            min_trip = trip
    return min_trip

firstGrouped = tripsByStartStation\
  .groupByKey()\
  .mapValues(lambda trips: earliestTrip(trips))

In [ ]:
%%time

firstGrouped.take(5)

CPU times: user 178 ms, sys: 19.6 ms, total: 197 ms
Wall time: 30 s


[('South Van Ness at Market',
  Trip(trip_id=4073, duration=1067, start_date=datetime.datetime(2013, 8, 29, 9, 24), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 9, 42), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=321, subscription_type='Subscriber', zip_code='94703')),
 ('Santa Clara at Almaden',
  Trip(trip_id=4500, duration=109, start_date=datetime.datetime(2013, 8, 29, 13, 25), start_station_name='Santa Clara at Almaden', start_station_id=4, end_date=datetime.datetime(2013, 8, 29, 13, 27), end_station_name='Adobe on Almaden', end_station_id='5', bike_id=679, subscription_type='Subscriber', zip_code='95112')),
 ('Clay at Battery',
  Trip(trip_id=4283, duration=1712, start_date=datetime.datetime(2013, 8, 29, 11, 55), start_station_name='Clay at Battery', start_station_id=41, end_date=datetime.datetime(2013, 8, 29, 12, 23), end_station_name='Harry Bridges Plaza (Ferry Building)',

In [ ]:
firstGrouped = tripsByStartStation\
  .reduceByKey(lambda tripA, tripB: tripA if tripA.start_date < tripB.start_date else tripB)

In [ ]:
%%time

firstGrouped.take(5)

CPU times: user 137 ms, sys: 30.5 ms, total: 168 ms
Wall time: 21.7 s


[('South Van Ness at Market',
  Trip(trip_id=4074, duration=1131, start_date=datetime.datetime(2013, 8, 29, 9, 24), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 9, 43), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=317, subscription_type='Subscriber', zip_code='94115')),
 ('Santa Clara at Almaden',
  Trip(trip_id=4500, duration=109, start_date=datetime.datetime(2013, 8, 29, 13, 25), start_station_name='Santa Clara at Almaden', start_station_id=4, end_date=datetime.datetime(2013, 8, 29, 13, 27), end_station_name='Adobe on Almaden', end_station_id='5', bike_id=679, subscription_type='Subscriber', zip_code='95112')),
 ('Clay at Battery',
  Trip(trip_id=4283, duration=1712, start_date=datetime.datetime(2013, 8, 29, 11, 55), start_station_name='Clay at Battery', start_station_id=41, end_date=datetime.datetime(2013, 8, 29, 12, 23), end_station_name='Harry Bridges Plaza (Ferry Building)',

In [ ]:
tripData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trips.csv")

tripData

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
4576,63,null,South Van Ness at...,66,2013-08-29 14:14:00,South Van Ness at...,66,520,Subscriber,94127
4607,null,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber,95138
4130,71,2013-08-29 10:16:00,Mountain View Cit...,27,2013-08-29 10:17:00,Mountain View Cit...,27,48,Subscriber,97214
4251,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber,95060
4299,83,2013-08-29 12:02:00,South Van Ness at...,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber,94103
4927,103,2013-08-29 18:54:00,Golden Gate at Polk,59,2013-08-29 18:56:00,Golden Gate at Polk,59,527,Subscriber,94109
4500,109,2013-08-29 13:25:00,Santa Clara at Al...,4,2013-08-29 13:27:00,Adobe on Almaden,5,679,Subscriber,95112
4563,111,2013-08-29 14:02:00,San Salvador at 1st,8,2013-08-29 14:04:00,San Salvador at 1st,8,687,Subscriber,95112
4760,113,2013-08-29 17:01:00,South Van Ness at...,66,2013-08-29 17:03:00,South Van Ness at...,66,553,Subscriber,94103
4258,114,2013-08-29 11:33:00,San Jose City Hall,10,2013-08-29 11:35:00,MLK Library,11,107,Subscriber,95060


In [ ]:
tripData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [ ]:
tripData.show(n=5)

+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|         start_date|  start_station_name|start_station_id|           end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|               null|South Van Ness at...|              66|2013-08-29 14:14:00|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|    null|2013-08-29 14:42:00|  San Jose City Hall|              10|2013-08-29 14:43:00|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|2013-08-29 10:16:00|Mountain View Cit...|              27|2013-08-29 10:17:00|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|    

In [ ]:
tripData.dropna().show(n=5)

+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|         start_date|  start_station_name|start_station_id|           end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|4130|      71|2013-08-29 10:16:00|Mountain View Cit...|              27|2013-08-29 10:17:00|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|      77|2013-08-29 11:29:00|  San Jose City Hall|              10|2013-08-29 11:30:00|  San Jose City Hall|            10|     26|       Subscriber|   95060|
|4299|      83|2013-08-29 12:02:00|South Van Ness at...|              66|2013-08-29 12:04:00|      Market at 10th|            67|    319|       Subscriber|   94103|
|4927|    

In [ ]:
tripData.describe().show()

+-------+-----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+--------------------+
|summary|               id|          duration|  start_station_name|  start_station_id|    end_station_name|    end_station_id|           bike_id|subscription_type|            zip_code|
+-------+-----------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+--------------------+
|  count|           669959|            669958|              669959|            669959|              669959|            669959|            669959|           669959|              663340|
|   mean|460382.0098991132| 1107.951395460611|                null| 57.85187601032302|                null|57.837437813358726|427.58761954089726|             null|  1576245.3147059095|
| stddev|264584.4584872604|22255.453593552545|                null|17.11247

In [ ]:
stationData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y')\
.csv("stations.csv")

stationData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: timestamp (nullable = true)



In [ ]:
stationData.show(n=5)

+---+--------------------+------------------+-------------------+----------+--------+-------------------+
| id|                name|               lat|               long|dock_count|    city|  installation_date|
+---+--------------------+------------------+-------------------+----------+--------+-------------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|San Jose|2013-08-06 00:00:00|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|San Jose|2013-08-05 00:00:00|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|San Jose|2013-08-06 00:00:00|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|San Jose|2013-08-05 00:00:00|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|San Jose|2013-08-07 00:00:00|
+---+--------------------+------------------+-------------------+----------+--------+-------------------+
only showing top 5 rows



In [ ]:
stationData.describe().show()

+-------+------------------+--------------------+-------------------+-------------------+-----------------+-------------+
|summary|                id|                name|                lat|               long|       dock_count|         city|
+-------+------------------+--------------------+-------------------+-------------------+-----------------+-------------+
|  count|                70|                  70|                 70|                 70|               70|           70|
|   mean|              43.0|                null|  37.59024338428572|-122.21841616428571|17.65714285714286|         null|
| stddev|24.166091947189145|                null|0.20347253639672502|0.20944604979644524|4.010441857493954|         null|
|    min|                 2|       2nd at Folsom|          37.329732|        -122.418954|               11|Mountain View|
|    max|                84|Yerba Buena Cente...|           37.80477|        -121.877349|               27|     San Jose|
+-------+---------------

In [ ]:
stationsView = stationData.select(stationData['id'], stationData['name'], stationData['lat'], stationData['long'])
stationsView.show()

+---+--------------------+------------------+-------------------+
| id|                name|               lat|               long|
+---+--------------------+------------------+-------------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|
|  7|Paseo de San Antonio|         37.333798|-121.88694299999999|
|  8| San Salvador at 1st|         37.330165|-121.88583100000001|
|  9|           Japantown|         37.348742|-121.89471499999999|
| 10|  San Jose City Hall|         37.337391|        -121.886995|
| 11|         MLK Library|         37.335885|-121.88566000000002|
| 12|SJSU 4th at San C...|         37.332808|-121.88389099999999|
| 13|       St James Park|         37.339301|-121.88993700000002|
| 14|Arena

In [ ]:
startTrips = tripData.select(tripData.id, tripData.duration, tripData.start_station_id).withColumnRenamed('id', 'trip_id').join(stationsView, tripData.start_station_id == stationsView.id)
startTrips = startTrips.drop('id')

In [ ]:
startTrips.show()

+-------+--------+----------------+--------------------+------------------+-------------------+
|trip_id|duration|start_station_id|                name|               lat|               long|
+-------+--------+----------------+--------------------+------------------+-------------------+
|   4576|      63|              66|South Van Ness at...|         37.774814|        -122.418954|
|   4607|    null|              10|  San Jose City Hall|         37.337391|        -121.886995|
|   4130|      71|              27|Mountain View Cit...|         37.389218|        -122.081896|
|   4251|      77|              10|  San Jose City Hall|         37.337391|        -121.886995|
|   4299|      83|              66|South Van Ness at...|         37.774814|        -122.418954|
|   4927|     103|              59| Golden Gate at Polk|         37.781332|        -122.418603|
|   4500|     109|               4|Santa Clara at Al...|         37.333988|        -121.894902|
|   4563|     111|               8| San 

In [ ]:
tripData.printSchema()
stationData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: timestamp (nullable = true)



In [ ]:
stationData.createOrReplaceTempView("stations")
tripData.createOrReplaceTempView("trips")

In [ ]:
endTrips = spark.sql("""
SELECT trips.id as trip_id, trips.end_station_id, trips.duration, stations.name as station_name, stations.lat, stations.long
FROM trips INNER JOIN stations
    ON trips.end_station_id==stations.id
""")
endTrips.show()

+-------+--------------+--------+--------------------+------------------+-------------------+
|trip_id|end_station_id|duration|        station_name|               lat|               long|
+-------+--------------+--------+--------------------+------------------+-------------------+
|   4576|            66|      63|South Van Ness at...|         37.774814|        -122.418954|
|   4607|            10|    null|  San Jose City Hall|         37.337391|        -121.886995|
|   4130|            27|      71|Mountain View Cit...|         37.389218|        -122.081896|
|   4251|            10|      77|  San Jose City Hall|         37.337391|        -121.886995|
|   4299|            67|      83|      Market at 10th|37.776619000000004|-122.41738500000001|
|   4927|            59|     103| Golden Gate at Polk|         37.781332|        -122.418603|
|   4500|             5|     109|    Adobe on Almaden|         37.331415|          -121.8932|
|   4563|             8|     111| San Salvador at 1st|      

In [ ]:
spark.sql("""
SELECT start_station_name, avg(duration)
FROM trips
GROUP BY trips.start_station_name
ORDER BY avg(duration) DESC
""").show()

+--------------------+------------------+
|  start_station_name|     avg(duration)|
+--------------------+------------------+
|University and Em...| 7090.239417989418|
|California Ave Ca...| 4628.005847953216|
|Redwood City Publ...| 4579.234741784037|
|       Park at Olive|4438.1613333333335|
|San Jose Civic Ce...| 4208.016938519448|
|Rengstorff Avenue...| 4174.082373782108|
|Redwood City Medi...| 3959.491961414791|
|Palo Alto Caltrai...|3210.6489815253435|
|San Mateo County ...|2716.7700348432054|
|    Broadway at Main|2481.2537313432836|
|Cowper at University|2477.2379912663755|
|Redwood City Calt...|2415.7175032175032|
|South Van Ness at...| 2401.603338509317|
|San Antonio Caltr...| 2377.497487437186|
|San Antonio Shopp...| 2285.981298129813|
|           Japantown| 2207.809947643979|
|Washington at Kea...|1979.3077445652175|
|Arena Green / SAP...|1963.9679144385027|
|SJSU 4th at San C...| 1962.280341880342|
|Castro Street and...|1868.3135135135135|
+--------------------+------------

In [ ]:
! pip install h3 h3_pyspark pandas folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00


In [ ]:
spark.sql("""
SELECT bike_id, start_date, end_date
FROM trips
WHERE
    start_date > make_timestamp(2014, 12, 25, 0, 0, 0)
    AND start_date <  make_timestamp(2014, 12, 26, 0, 0, 0)
""").show()

+-------+-------------------+-------------------+
|bike_id|         start_date|           end_date|
+-------+-------------------+-------------------+
|    379|2014-12-25 22:10:00|2014-12-25 22:18:00|
|    709|2014-12-25 21:21:00|2014-12-25 21:27:00|
|    376|2014-12-25 20:40:00|2014-12-25 20:46:00|
|    541|2014-12-25 20:27:00|2014-12-25 20:32:00|
|    283|2014-12-25 19:56:00|2014-12-25 20:01:00|
|    519|2014-12-25 19:56:00|2014-12-25 20:01:00|
|    583|2014-12-25 19:05:00|2014-12-25 19:07:00|
|    495|2014-12-25 18:42:00|2014-12-25 18:44:00|
|    541|2014-12-25 18:28:00|2014-12-25 18:37:00|
|    585|2014-12-25 18:27:00|2014-12-25 18:37:00|
|    574|2014-12-25 18:12:00|2014-12-25 18:21:00|
|    630|2014-12-25 18:12:00|2014-12-25 18:22:00|
|    583|2014-12-25 18:05:00|2014-12-25 18:22:00|
|    290|2014-12-25 18:01:00|2014-12-25 18:15:00|
|    451|2014-12-25 17:55:00|2014-12-25 18:04:00|
|    630|2014-12-25 17:55:00|2014-12-25 17:59:00|
|    574|2014-12-25 17:54:00|2014-12-25 17:59:00|


In [ ]:
spark.sql("""
SELECT trips.bike_id, trips.start_date, trips.end_date, stations.name
FROM trips INNER JOIN stations
    ON trips.start_station_id == stations.id
WHERE
    bike_id == 583
    AND start_date > make_timestamp(2014, 12, 25, 0, 0, 0)
    AND start_date <  make_timestamp(2014, 12, 26, 0, 0, 0)
""").show()

+-------+-------------------+-------------------+--------------+
|bike_id|         start_date|           end_date|          name|
+-------+-------------------+-------------------+--------------+
|    583|2014-12-25 19:05:00|2014-12-25 19:07:00|Market at 10th|
|    583|2014-12-25 18:05:00|2014-12-25 18:22:00|Market at 10th|
|    583|2014-12-25 12:14:00|2014-12-25 12:21:00| Market at 4th|
+-------+-------------------+-------------------+--------------+



In [ ]:
from pyspark.sql import functions as F
import h3_pyspark
import h3

In [ ]:
resolution = 8
stationData.withColumn('h3', h3_pyspark.geo_to_h3('lat', 'long', F.lit(resolution))).createOrReplaceTempView("stations_h3")

In [ ]:
christmas_583_contacts = spark.sql("""
SELECT trips.bike_id, trips.start_date, stations_h3.h3, stations_h3.lat, stations_h3.long, stations_h3.name
FROM trips INNER JOIN stations_h3
    ON trips.start_station_id == stations_h3.id
WHERE
    stations_h3.h3 in (SELECT stations_h3.h3
                            FROM trips INNER JOIN stations_h3
                                ON trips.start_station_id == stations_h3.id
                            WHERE
                                bike_id == 583
                                AND start_date > make_timestamp(2014, 12, 25, 0, 0, 0)
                                AND start_date <  make_timestamp(2014, 12, 26, 0, 0, 0))
    AND start_date > make_timestamp(2014, 12, 25, 0, 0, 0)
    AND start_date < make_timestamp(2014, 12, 26, 0, 0, 0)
ORDER BY trips.start_date
""")
christmas_583_contacts.cache()
christmas_583_contacts.show()

+-------+-------------------+---------------+------------------+-------------------+------------------+
|bike_id|         start_date|             h3|               lat|               long|              name|
+-------+-------------------+---------------+------------------+-------------------+------------------+
|    439|2014-12-25 01:40:00|8828308281fffff|37.776619000000004|-122.41738500000001|    Market at 10th|
|    659|2014-12-25 09:49:00|88283082abfffff|37.781752000000004|-122.40512700000001|     5th at Howard|
|    465|2014-12-25 09:49:00|88283082abfffff|37.781752000000004|-122.40512700000001|     5th at Howard|
|    583|2014-12-25 12:14:00|88283082abfffff|         37.786305|-122.40496599999999|     Market at 4th|
|    479|2014-12-25 12:22:00|88283082abfffff|37.781752000000004|-122.40512700000001|     5th at Howard|
|    331|2014-12-25 12:22:00|88283082abfffff|37.781752000000004|-122.40512700000001|     5th at Howard|
|    330|2014-12-25 12:27:00|88283082abfffff|37.781752000000004|

In [ ]:
import pandas as pd
import h3

h3_places = christmas_583_contacts.select('lat','long', 'name', 'h3').toPandas()

In [ ]:
# source code from https://nbviewer.org/github/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb
import folium

def init_map(hexagons, width=1100, height=900):
    lats = []
    longs = []
    for hexagon in hexagons:
        lat, long = h3.h3_to_geo(hexagon)
        lats.append(lat)
        longs.append(long)
    return folium.Map(location=[sum(lats)/len(lats), sum(longs)/len(longs)], zoom_start=15, tiles='cartodbpositron', width=width, height=height)

def visualize_hexagons(folium_map, hexagons, color="red"):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons.
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0], polyline))
        lng.extend(map(lambda v:v[1], polyline))
        polylines.append(polyline)

    for polyline in polylines:
        my_PolyLine = folium.PolyLine(locations=polyline, weight=8, color=color)
        folium_map.add_child(my_PolyLine)

    return folium_map

def visualize_stations(folium_map, stations, color="red"):
    """
    stations is a dataframe with columns: lat, long, station_name
    """
    for idx, lat, long, station_name in stations.itertuples():
        folium_map.add_child(folium.map.Marker(location=(lat, long)))
        folium_map.add_child(folium.map.Marker(location=(lat, long),
                        icon=folium.features.DivIcon(
                          icon_size=(500,36),
                          icon_anchor=(-17,37),
                          html=f'<div style="display: inline-block;font-size: 10pt; background: rgba(255, 255, 255, 0.8)">{station_name}</div>',
        )))
    return folium_map

In [ ]:
m = init_map(h3_places.h3.unique())
visualize_hexagons(m, h3_places.h3.unique(), color="black")
visualize_stations(m, h3_places.loc[:, ['lat', 'long', 'name']])
display(m)